In [143]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [128]:
s = requests.Session()
s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'

In [129]:
r = s.get('http://www.realtor.com/realestateandhomes-search/Bloomington_IL?pgsz=50')

In [130]:
data = r.text
soup = BeautifulSoup(data, 'lxml')
last_page_number = int(soup.find_all("span", class_="page")[-1].text.strip())

In [140]:
def get_homes_on_page(page_number):
    url = 'http://www.realtor.com/realestateandhomes-search/Bloomington_IL/pg-' + str(page_number) +'?pgsz=50' 
    page = s.get(url)
    data = page.text
    soup = BeautifulSoup(data, 'lxml')
    house_list = []
    houses = soup.find_all("div", class_="srp-item")
    for house in houses:
    
        try:
            house_listing = {
            'price':house.attrs['data-price'],
            'agent':house.attrs['data-advertiserid-agent'],
            'broker':house.attrs['data-advertiserid-broker'],
            'baths_full':house.attrs['data-baths_full'],
            'baths_half':house.attrs['data-baths_half'],
            'lot_size':house.attrs['data-lot_size'],
            'address':house.find("span", attrs={'itemprop': 'streetAddress'}).text,
            'city':house.find("span", attrs={'itemprop': 'addressLocality' }).text,
            'zip_code':house.find("span", attrs={'itemprop': 'postalCode' }).text,
            'photos':house.find("span", attrs={'data-label': 'property-photo-count' }).text,
            'property_type':house.find("span", attrs={'class': 'srp-property-type' }).text,
            'bedrooms':house.find("span", attrs={'class': 'data-value meta-beds' }).text,
            'sqft':house.find("li", attrs={'data-label': 'property-meta-sqft' }).text,
            'garage':house.find("li", attrs={'data-label': 'property-meta-garage' }).text
            }
            house_list.append(house_listing)
        except Exception as e:
            pass
        
    
    return house_list

In [141]:
houses = []
for page in range(1, last_page_number+1):    
    houses.extend(get_homes_on_page(page_number=page))

In [142]:
houses

[{'address': '36 Pembrook Cir',
  'agent': '1231308',
  'baths_full': '2',
  'baths_half': '1',
  'bedrooms': '4',
  'broker': '1295017',
  'city': 'Bloomington',
  'garage': '2 car',
  'lot_size': '',
  'photos': '30',
  'price': '169900',
  'property_type': 'Single Family Home',
  'sqft': '2,913 sq ft',
  'zip_code': '61704'},
 {'address': '35 Pembrook Cir',
  'agent': '7995',
  'baths_full': '2',
  'baths_half': '2',
  'bedrooms': '4',
  'broker': '3054550',
  'city': 'Bloomington',
  'garage': '2 car',
  'lot_size': '',
  'photos': '30',
  'price': '197000',
  'property_type': 'Single Family Home',
  'sqft': '2,546 sq ft',
  'zip_code': '61704'},
 {'address': '3 Litta Ct',
  'agent': '1295102',
  'baths_full': '3',
  'baths_half': '1',
  'bedrooms': '5',
  'broker': '1295017',
  'city': 'Bloomington',
  'garage': '3 car',
  'lot_size': '',
  'photos': '30',
  'price': '379900',
  'property_type': 'Single Family Home',
  'sqft': '3,780 sq ft',
  'zip_code': '61704'},
 {'address': '2

In [145]:
house_df = pd.DataFrame.from_records(houses)

In [146]:
house_df.to_csv("house_data.csv", index=False)